# Part 0 - Data Upload

In [1]:
from google.cloud.bigquery import Client, QueryJobConfig

client = Client()

query = """
SELECT 
    author_id, 
    created_at, 
    text, 
    id 
FROM `fake-news-bears.usa_congress_twitter.tweets`
WHERE text != ""
"""
job = client.query(query)
test_df = job.to_dataframe()

In [ ]:
#from google.cloud.bigquery import Client, QueryJobConfig

#client = Client()

#query = """
#SELECT
#    user_id, 
#    text
#FROM `fake-news-bears.twitter_2016.2016_first_pres_debate`
#WHERE created_at BETWEEN "2016-09-26 00:00:00 UTC" and "2016-09-26 01:30:00 UTC"
#limit 100;
#"""
#job = client.query(query)
#test_df = job.to_dataframe()

In [2]:
test_df

,author_id,created_at,text,id
0,1002630999052865536,2023-01-25 19:16:37+00:00,ICYMI: House Republicans Release Economic Plan...,1618326996916011008
1,1002630999052865536,2023-01-01 17:49:18+00:00,Happy New Year! No matter the obstacles we fac...,1609607710986608641
2,1004891731,2022-12-22 15:14:48+00:00,Inflation continues to ravage the paychecks of...,1605944953095454721
3,1004891731,2022-11-24 15:46:19+00:00,"Happy Thanksgiving to all celebrating, and a s...",1595806025080528898
4,1004891731,2022-11-07 17:11:58+00:00,"Americans don't deserve record inflation, high...",1589666984157143040
...,...,...,...,...
49565,234374703,2022-12-22 23:55:00+00:00,GREAT NEWS: I secured more than $228 MILLION f...,1606075864709414917
49566,29201047,2023-02-02 17:35:10+00:00,I\u2019m heading out on the road to meet with ...,1621200569498013697
49567,515822213,2023-01-13 19:30:44+00:00,Had a great week meeting with Montanans face-t...,1613981892419719171
49568,5558312,2023-01-06 18:49:15+00:00,"With POTUS' signature yesterday, the Butterfie...",1611434738090221568


# Part 1 - LSTM

In [3]:
!pip install wrapt
!pip install opt_einsum
!pip install gast
!pip install astunparse
!pip install termcolor
!pip install flatbuffers
!pip install keras
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_text
!pip install nltk
!pip install gensim


  Using cached wrapt-1.15.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (75 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires astunparse>=1.6.0, which is not installed.
tensorflow 2.11.0 requires flatbuffers>=2.0, which is not installed.
tensorflow 2.11.0 requires gast<=0.4.0,>=0.2.1, which is not installed.
tensorflow 2.11.0 requires google-pasta>=0.1.1, which is not installed.
tensorflow 2.11.0 requires h5py>=2.9.0, which is not installed.
tensorflow 2.11.0 requires keras<2.12,>=2.11.0, which is not installed.
tensorflow 2.11.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.11.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.11.0 requires tensorboard<2.12,>=2.11, which is not installed.
tensorflow 2.11.0 requires tensorflow-estimator<2.12,>=2.1

In [4]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import seaborn as sns


2023-02-27 14:59:32.105626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 14:59:46.039329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-27 14:59:46.040331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [ ]:
lstm_model = open('./lstm.pkl', "rb")
lstm = pickle.load(lstm_model)

Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-02-08 20:21:10           64
config.json                                    2023-02-08 20:21:10         3048
variables.h5                                   2023-02-08 20:21:10       768000


2023-02-27 08:53:32.237323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 08:53:33.017250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 08:53:33.017582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 08:53:33.050385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [ ]:
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [ ]:
val = []

sample = np.array([['vaccines','do','not','work']])
for i in sample:
    temp = np.array(embed(i))
    val.append(temp)

In [ ]:
val = np.array(val)

holder = lstm.predict(val)
holder[0]

2023-02-27 07:18:25.628176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


1/1 [==============================] - 27s 27s/step


array([0.3787054], dtype=float32)

In [ ]:
holder[0][0]

0.3787054

In [ ]:
def predict_test(tweet_array):
    
    output_array = []
    
    for tweet in tweet_array:
        val = []

        for i in sample:
            print(i)
            temp = np.array(embed(i))
            val.append(temp)

        val = np.array(val)

        output_array.append(lstm.predict(val)[0][0])



In [ ]:
np.array(['trump'])

array(['trump'], dtype='<U5')

In [ ]:
np.array([['vaccines','do','not','work']])

array([['vaccines', 'do', 'not', 'work']], dtype='<U8')

In [ ]:
for i in sample:
    print(i)

['vaccines' 'do' 'not' 'work']


In [ ]:
sample = np.array([['vaccines','do','not','work']])

predict_test(sample)

['vaccines' 'do' 'not' 'work']
1/1 [==============================] - 0s 21ms/step


In [ ]:
np.array(embed(['trump']))


array([[-0.06012718, -0.05494037,  0.02107424,  0.01254636, -0.00696223,
        -0.03486495, -0.02314042, -0.09726806,  0.00919119,  0.01728892,
        -0.03742978,  0.04518568, -0.03743932,  0.13630173, -0.002566  ,
        -0.05437953,  0.08267361,  0.01209148, -0.01561307,  0.09825669,
         0.16592625, -0.00589087, -0.01194644, -0.00224337,  0.04272262,
         0.11662748,  0.04387851, -0.02188307, -0.02613572, -0.01605892,
         0.12024305, -0.02570795,  0.0307494 ,  0.02620539, -0.02943143,
         0.05880465, -0.08816493,  0.02910838, -0.04380202, -0.04556799,
        -0.05387823, -0.15247203, -0.03076144, -0.02202726, -0.03232737,
         0.02784347, -0.02141403,  0.00306806,  0.03168983,  0.02787551,
         0.00025703,  0.06051562, -0.03029486,  0.20816961,  0.01402717,
        -0.01496646, -0.00347941,  0.05895738,  0.04540522,  0.01963709,
         0.05724182, -0.012306  , -0.08218483,  0.06494712,  0.02134916,
        -0.00849398, -0.00618139, -0.03063439, -0.0

In [ ]:
import re 

def predict(tweet_array):

    word_array = np.array(tweet_array.split(" "))
    #print(word_array)

    val = []

    for word in word_array:
        new_word = re.sub(r'[^A-Za-z0-9 ]+', '', word)

        #print(word)

        temp = np.array(embed([new_word]))
        #print(temp)
        val.append(temp)


    val = np.array(val)



    holder = lstm.predict(val)
        

    return holder[0]
        #return holder[0]

In [ ]:
#next step - just ened to be able to append value to the column - basically how to go down
#a dataframe and add a row at a time to each cell

In [ ]:
test_df['lstm_score'] = test_df.apply(lambda row: predict(row['text']), axis=1)


2023-02-27 08:54:03.646468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


2/2 [==============================] - 0s 4ms/step


In [ ]:
test_df.to_csv('lstm_model_scores_appended_politicians.csv')

In [ ]:
test_df

,user_id,text,lstm_score
0,771135164797902852,trump,[0.4674162]
1,1494027864,@HillaryClinton @Aroyaldmd I agree! I stand by...,[0.4763733]
2,779865353895735296,#debates,[0.35284382]
3,723520270976188417,Go Trump.,[0.573256]
4,91873338,SEXXXXTOU,[0.4763733]
...,...,...,...
95,528244909,https://t.co/q0TMBEH5RF,[0.4763733]
96,3074664830,https://t.co/q8TJpN6HHr,[0.4763733]
97,144189766,https://t.co/rdpHthPRZZ,[0.4763733]
98,720029805597429760,https://t.co/rkgrRuQ618,[0.4763733]


# Part 2 - Dense Model

Functional, but seems to assign every tweet a very low truthiness score

In [5]:
from tensorflow import keras
dense_model= keras.models.load_model('./densemodel/')
import nltk


2023-02-27 14:59:55.472023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 14:59:56.521953: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 14:59:56.522287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-27 14:59:56.542801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
doc_file = open('./doc.pkl', "rb")
doc = pickle.load(doc_file)

#lstm_model = open('./lstm.pkl', "rb")


In [7]:
import string 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#Algorithm used to preprocess a new sentence
sentence = "California governor says he will use legal tactics of Texas abortion ban to implement gun control"

def preprocess_dense(sentence):
    #New sentence for the model

    #Lowercase words
    sentence = sentence.lower()

    #Ensure that all necessary punctuations are in one list
    #Include ' and " as they are not default
    punc = list(string.punctuation)
    punc.append('\'')
    punc.append('"')
    #print(punc)

    #Loop through sentence and remove all punctuations
    for i in string.punctuation:
        sentence = sentence.replace(i, '')

    #Tokenize sentence -> all words in a new list
    tok = sentence.split(' ')

    #Define text lemmatization model (eg: walks will be changed to walk)
    lemmatizer = WordNetLemmatizer()

    #Lemmatize each word in the sentence
    for w in sentence:
        lemmatizer.lemmatize(sentence)


    #Define all stopwords in the English language (it, was, for, etc.)
    stop = stopwords.words('english')




    #Remove them from our dataframe and store in a new list
    minStop = []

    for i in tok:
      if i not in stop:
        minStop.append(i)

    #Doc2Vec tags
    tag = [TaggedDocument(minStop,[0])]

    predVec = [doc.infer_vector(minStop)]
    predVec = np.array(predVec)

    return predVec



[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:


#Import Doc2Vec to get a vector for each individual title
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


""" Defunct code?


#Add a tag which denotes a new document everytime a new dataset is seen
#Example: [[word1,word2,word3,word4],document0]
tags = [TaggedDocument(doc, [i]) for i, doc in enumerate(title_train)]

#Define the Doc2Vec model
doc = Doc2Vec(tags,vector_size=100,min_count=1,epochs=5)
#Build the vocabulary based on the tag structures defined above
#doc.build_vocab(tags)
#Train the doc2vec model
doc.train(tags, total_examples=doc.corpus_count,epochs=doc.epochs)

""" 

NameError: name 'title_train' is not defined

In [9]:
def dense_predict(tweet_array):
    word_array = tweet_array
    val = preprocess_dense(word_array)
    holder = dense_model.predict(val)

    return holder[0]

In [ ]:
test_df['dense_model_score'] = test_df.apply(lambda row: dense_predict(row['text']), axis=1)


1/1 [==============================] - ETA: 0s

In [ ]:
test_df.to_csv('dense_layer_model_scores_appended_politicians.csv')

In [ ]:
test_df

,user_id,text,lstm_score,dense_model_score
0,771135164797902852,trump,[0.4674162],[0.21758018]
1,1494027864,@HillaryClinton @Aroyaldmd I agree! I stand by...,[0.4763733],[0.0627452]
2,779865353895735296,#debates,[0.35284382],[0.1675138]
3,723520270976188417,Go Trump.,[0.573256],[0.21781297]
4,91873338,SEXXXXTOU,[0.4763733],[0.15414155]
...,...,...,...,...
95,528244909,https://t.co/q0TMBEH5RF,[0.4763733],[0.13651904]
96,3074664830,https://t.co/q8TJpN6HHr,[0.4763733],[0.15332267]
97,144189766,https://t.co/rdpHthPRZZ,[0.4763733],[0.16889362]
98,720029805597429760,https://t.co/rkgrRuQ618,[0.4763733],[0.0930751]


# Part 3 - BERT Model

Doesn't load correctly from pickle - afraid we'll have to re-run and try to save model a different way

In [ ]:
bert_model = open('./bert.pkl', "rb")
bert = pickle.load(bert_model)

Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-02-12 00:32:56           64
config.json                                    2023-02-12 00:32:56         2418
variables.h5                                   2023-02-12 00:32:56    438125720


ValueError: Unknown layer: 'KerasLayer'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
# jason's code for bigquery loading

table_id = "fake-news-bears.teamwork.jyang_political_sentiment"


job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("author_id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("text", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        # bigquery.SchemaField("negative", bigquery.enums.SqlTypeNames.FLOAT),
        # bigquery.SchemaField("neutral", bigquery.enums.SqlTypeNames.FLOAT),
        # bigquery.SchemaField("positive", bigquery.enums.SqlTypeNames.FLOAT),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

